In [9]:
import pandas as pd
import re
import os
import glob

In [10]:
def add_names(df_articles, df_sections):
    deel_naam = ''
    df_articles['name']= df_articles['name'].str.replace('^Artikel [^\s]+ ', '', regex=True)
    df_articles.columns=['art_nr', 'article', 'subdivision']
    columns = ['art_nr', 'article', 'subdivision', 'codex', 'part', 'book', 'title', 'chapter', 'section', 'subsection', 'part_name', 'book_name', 'title_name', 'chapter_name', 'section_name', 'subsection_name']
    df_articles[['codex', 'book', 'title', 'chapter', 'section', 'subsection']] = df_articles['subdivision'].str.split(',', expand=True)
    if '--' in df_articles['book'].iloc[0]:
        df_articles[['part', 'book']] = df_articles['book'].str.split('--', 1, expand=True)
        deel_naam = df_articles['part'].iloc[0].strip()
        if '-' in df_articles['part'].iloc[0]:
            df_articles['part'] = df_articles['part'].str.split('-', 1).str[0]
    else:
        df_articles['part'] = ''
    df_articles = df_articles.reindex(columns=columns[:10])
    df_sections = df_sections.iloc[:, :-1]
    df_sections['name'] = df_sections['name'].apply(lambda x: re.sub('^[^a-zA-Z(]*', '', x) if pd.notna(x) else x)

    dict_subdivisions = {', , , , ': '-#--#--#--#-'}
    
    if deel_naam != '':
        deel = (deel_naam, deel_naam)
        dict_subdivisions[deel_naam + '--' + ', , , , '] = deel_naam + '---' + '-#--#--#--#-'
    else:
        deel = ('', '')
    boek = ('', '')
    titel = ('', '')
    hoofdstuk = ('', '')
    afdeling = ('', '')
    onderafdeling = ('', '')

    def create_key(d=('',''), b=('',''), t=('',''), h=('',''), a=('',''), o=('','')):
        nonlocal deel, boek, titel, hoofdstuk, afdeling, onderafdeling
        deel = d
        boek = b
        titel = t
        hoofdstuk = h
        afdeling = a 
        onderafdeling = o
        key_deel = deel[0] + '--' if deel[0] != '' else ''
        value_deel = deel[1] + '---' if deel[0] != '' else ''
        key_suffix = ', '.join([boek[0], titel[0], hoofdstuk[0], afdeling[0], onderafdeling[0]])
        value_suffix = '-#-'.join([boek[1], titel[1], hoofdstuk[1], afdeling[1], onderafdeling[1]])
        dict_subdivisions[key_deel + key_suffix] = value_deel + value_suffix
    
    for index, row in df_sections.iterrows():
        title = '' if pd.isna(row['title']) else row['title']
        name = '' if pd.isna(row['name']) else row['name']
        if 'deel' in title.lower():
            create_key((title, name))
        elif 'boek' in title.lower():
            create_key(deel, (title, name))
        elif 'titel' in title.lower():
            create_key(deel, boek, (title, name))
        elif 'hoofdstuk' in title.lower():
            create_key(deel, boek, titel, (title, name))
        elif 'onderafdeling' in title.lower():
            create_key(deel, boek, titel, hoofdstuk, afdeling, (title, name))
        elif 'afdeling' in title.lower():
            create_key(deel, boek, titel, hoofdstuk, (title, name))

    try:
        df_articles['subdivision_names'] = df_articles['subdivision'].apply(lambda x: dict_subdivisions[x.split(', ', 1)[1]])
        if '---' in df_articles['subdivision_names'].iloc[0]:
            df_articles[['part_name', 'subdivision_names']] = df_articles['subdivision_names'].str.split('---', 1, expand=True)
        else:
            df_articles['part_name'] = ''
        df_articles[columns[-5:]] = df_articles['subdivision_names'].str.split('-#-', expand=True)
        df_articles.drop('subdivision_names', axis=1, inplace=True)

    except KeyError as e:
        print(f"KeyError occurred: {e}")
        print(dict_subdivisions.keys())
        return False
    return df_articles





In [11]:
df_bsard_dutch = pd.DataFrame()
for file_path in glob.glob('csv_scraped/*_articles.csv'):
    filename = re.search('^(.*?)\_articles\.csv', os.path.basename(file_path)).group(1)
    df_articles = pd.read_csv(file_path)
    df_sections = pd.read_csv('csv_scraped\\' + filename + '_sections.csv')
    # Process each file
    print("Processing file:", file_path)
    df_bsard_dutch = df_bsard_dutch.append(add_names(df_articles, df_sections))
df_bsard_dutch

Processing file: csv_scraped\boswetboek_articles.csv
Processing file: csv_scraped\br_huisvestingscode_articles.csv
Processing file: csv_scraped\br_lucht_klimaat_energie_articles.csv
Processing file: csv_scraped\br_wetboek_ruimtelijke_ordening_articles.csv
Processing file: csv_scraped\codex_welzijn_werk_articles.csv
Processing file: csv_scraped\consulair_wetboek_articles.csv
Processing file: csv_scraped\democratie_articles.csv
Processing file: csv_scraped\diverse_rechten_en_taksen_articles.csv
Processing file: csv_scraped\economisch_recht_articles.csv
Processing file: csv_scraped\gerechtelijk_wetboek_algemene_beginselen_articles.csv
Processing file: csv_scraped\gerechtelijk_wetboek_deel_2_articles.csv
Processing file: csv_scraped\gerechtelijk_wetboek_deel_3_articles.csv
Processing file: csv_scraped\gerechtelijk_wetboek_deel_4_articles.csv
Processing file: csv_scraped\gerechtelijk_wetboek_deel_5_articles.csv
Processing file: csv_scraped\gerechtelijk_wetboek_deel_6_articles.csv
Processing

C:\Users\franc\AppData\Local\Temp\ipykernel_33392\2958284747.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sections['name'] = df_sections['name'].apply(lambda x: re.sub('^[^a-zA-Z(]*', '', x) if pd.notna(x) else x)


Processing file: csv_scraped\gerechtelijk_wetboek_deel_9_bijvoegsel_articles.csv
Processing file: csv_scraped\grondwet_articles.csv
Processing file: csv_scraped\kieswetboek_articles.csv
Processing file: csv_scraped\oud_1_personen_articles.csv
Processing file: csv_scraped\oud_3_eigendomsverkrijging_2_articles.csv
Processing file: csv_scraped\oud_3_eigendomsverkrijging_3_articles.csv
Processing file: csv_scraped\oud_3_eigendomsverkrijging_4_articles.csv
Processing file: csv_scraped\privaatrecht_articles.csv
Processing file: csv_scraped\reg_basis_secundair_onderwijs_articles.csv
Processing file: csv_scraped\reg_dierenwelzijn_articles.csv
Processing file: csv_scraped\reg_duurzaam_wonen_articles.csv
Processing file: csv_scraped\reg_landbouwwetboek_articles.csv
Processing file: csv_scraped\reg_milieuwetboek_boek_1_articles.csv
Processing file: csv_scraped\reg_ruimtelijke_ontwikkeling_decr_articles.csv
Processing file: csv_scraped\reg_ruimtelijke_ontwikkeling_regl_articles.csv
Processing file

art_nr                                            article  \
0        Art.1  Onder de bosregeling vallen en overeenkomstig ...   
1     Art.1bis  (WAALSE GEWEST) In het Waalse Gewest kunnen de...   
2        Art.2  Van de bepalingen van artikel 1 zijn uitgezond...   
3        Art.3  Bossen die aan bijzondere personen toebehoren,...   
4        Art.4  De organisatie van het bosbeheer, de wijze van...   
...        ...                                                ...   
1231  Art.18:4  Een lid van een Europees economisch samenwerki...   
1232  Art.18:5  Het Europees economisch samenwerkingsverband m...   
1233  Art.18:6  Niettegenstaande andersluidende bepaling in de...   
1234  Art.18:7  De ondernemingen die over een ondernemingsraad...   
1235  Art.18:8  In afwijking van de artikelen 1:5, § 3, en 2:6...   

                                            subdivision  \
0                           Boswetboek, , Titel 1, , ,    
1                           Boswetboek, , Titel 1, , ,    
2                           Boswetboek, , Titel 1, , ,    
3                           Boswetboek, , Titel 1, , ,    
4                           Boswetboek, , Titel 2, , ,    
...                                                 ...   
1231  Wetboek van vennootschappen en verenigingen, D...   
1232  Wetboek van vennootschappen en verenigingen, D...   
1233  Wetboek van vennootschappen en verenigingen, D...   
1234  Wetboek van vennootschappen en verenigingen, D...   
1235  Wetboek van vennootschappen en verenigingen, D...   

                                            codex     part     book     title  \
0                                      Boswetboek                     Titel 1   
1                                      Boswetboek                     Titel 1   
2                                      Boswetboek                     Titel 1   
3                                      Boswetboek                     Titel 1   
4                                      Boswetboek                     Titel 2   
...                                           ...      ...      ...       ...   
1231  Wetboek van vennootschappen en verenigingen   DEEL 5  BOEK 18   TITEL 1   
1232  Wetboek van vennootschappen en verenigingen   DEEL 5  BOEK 18   TITEL 1   
1233  Wetboek van vennootschappen en verenigingen   DEEL 5  BOEK 18   TITEL 1   
1234  Wetboek van vennootschappen en verenigingen   DEEL 5  BOEK 18   TITEL 2   
1235  Wetboek van vennootschappen en verenigingen   DEEL 5  BOEK 18   TITEL 3   

           chapter section subsection                 part_name  \
0                                                                 
1                                                                 
2                                                                 
3                                                                 
4                                                                 
...            ...     ...        ...                       ...   
1231   HOOFDSTUK 2                     De Europese rechtsvormen   
1232   HOOFDSTUK 3                     De Europese rechtsvormen   
1233   HOOFDSTUK 4                     De Europese rechtsvormen   
1234                                   De Europese rechtsvormen   
1235                                   De Europese rechtsvormen   

                                         book_name  \
0                                                    
1                                                    
2                                                    
3                                                    
4                                                    
...                                            ...   
1231  Het Europees economisch samenwerkingsverband   
1232  Het Europees economisch samenwerkingsverband   
1233  Het Europees economisch samenwerkingsverband   
1234  Het Europees economisch samenwerkingsverband   
1235  Het Europees economisch samenwerkingsverband   

                                           

In [12]:
df_bsard_dutch.to_csv('df_bsard_dutch.csv', index= False)